In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lenskit as lk 
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als, item_knn as knn
from lenskit import topn
import networkx as nx
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori,association_rules
import warnings
warnings.filterwarnings("ignore")

In [2]:
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv', encoding='latin-1')

FileNotFoundError: [Errno 2] No such file or directory: 'ratings.csv'

In [ ]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
2495,109487,Interstellar (2014),Sci-Fi|IMAX
2496,111362,X-Men: Days of Future Past (2014),Action|Adventure|Sci-Fi
2497,111759,Edge of Tomorrow (2014),Action|Sci-Fi|IMAX
2498,112556,Gone Girl (2014),Drama|Thriller


In [ ]:
print(ratings.shape)
ratings.head()

(264505, 4)


,userId,movieId,rating,timestamp
0,12882,1,4.0,1147195252
1,12882,32,3.5,1147195307
2,12882,47,5.0,1147195343
3,12882,50,5.0,1147185499
4,12882,110,4.5,1147195239


**Association Rule Based**

In [ ]:
ratings = ratings.pivot(index = 'movieId',  columns ='userId', values = 'rating')
ratings.head()

userId,316,320,359,370,910,975,1015,1387,1447,1588,...,137118,137209,137227,137446,137559,137609,137805,138072,138176,138200
movieId,,,,,,,,,,,,,,,,,,,,,
1,2.5,NaN,5.0,4.5,5.0,NaN,4.0,3.0,4.5,4.5,...,3.0,NaN,NaN,NaN,3.5,NaN,4.0,3.0,NaN,3.0
2,NaN,NaN,NaN,4.0,4.0,NaN,3.0,3.0,NaN,NaN,...,NaN,3.5,NaN,3.5,NaN,NaN,2.0,1.0,NaN,NaN
3,NaN,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,3.5,NaN,3.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
ratings = ratings.rename(index=movies.set_index('movieId')['title'])
ratings = ratings.transpose()
ratings

movieId,Toy Story (1995),Jumanji (1995),Grumpier Old Men (1995),Waiting to Exhale (1995),Father of the Bride Part II (1995),Heat (1995),Sabrina (1995),Sudden Death (1995),GoldenEye (1995),"American President, The (1995)",...,"Hunger Games: Catching Fire, The (2013)","Hobbit: The Desolation of Smaug, The (2013)","Wolf of Wall Street, The (2013)",Her (2013),"Grand Budapest Hotel, The (2014)",Interstellar (2014),X-Men: Days of Future Past (2014),Edge of Tomorrow (2014),Gone Girl (2014),Guardians of the Galaxy (2014)
userId,,,,,,,,,,,,,,,,,,,,,
316,2.5,NaN,NaN,NaN,NaN,NaN,2.0,NaN,2.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
359,5.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,4.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
370,4.5,4.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,2.5,3.0,4.5,4.0,NaN,NaN,3.0,4.5,3.5,3.0
910,5.0,4.0,3.5,NaN,3.5,3.5,NaN,NaN,NaN,4.0,...,NaN,NaN,3.5,NaN,NaN,NaN,NaN,4.5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137609,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
137805,4.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,1.5,3.0,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
138072,3.0,1.0,1.0,NaN,NaN,NaN,NaN,1.5,3.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
for i in range(len(ratings.columns)):
    for j in range(len(ratings.index)):
        if ratings.iloc[j,i] >= 3:
            ratings.iloc[j,i] = 1
        else:
            ratings.iloc[j,i] = 0

In [ ]:
ratings

movieId,Toy Story (1995),Jumanji (1995),Grumpier Old Men (1995),Waiting to Exhale (1995),Father of the Bride Part II (1995),Heat (1995),Sabrina (1995),Sudden Death (1995),GoldenEye (1995),"American President, The (1995)",...,"Hunger Games: Catching Fire, The (2013)","Hobbit: The Desolation of Smaug, The (2013)","Wolf of Wall Street, The (2013)",Her (2013),"Grand Budapest Hotel, The (2014)",Interstellar (2014),X-Men: Days of Future Past (2014),Edge of Tomorrow (2014),Gone Girl (2014),Guardians of the Galaxy (2014)
userId,,,,,,,,,,,,,,,,,,,,,
316,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
320,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
359,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
370,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0
910,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137609,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
137805,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
138072,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#make dictionary of user and their movies
user_movies = {}
for i in range(len(ratings.index)):
    user_movies[ratings.index[i]] = []
    for j in range(len(ratings.columns)):
        if ratings.iloc[i,j] != 0:
            user_movies[ratings.index[i]].append(ratings.columns[j])

In [ ]:
#delet user id and put lists in numpy array from user_movies
user_movies2 = np.array(list(user_movies.values()))

In [ ]:
user_movies3 = []
for i in range(len(user_movies2)-1):
    if len(user_movies2[i]) < 40:
        user_movies3.append(user_movies2[i])

In [ ]:
te = TransactionEncoder()
te_ary = te.fit(user_movies3).transform(user_movies3)
dataset = pd.DataFrame(te_ary, columns=te.columns_)
dataset

,"'burbs, The (1989)",(500) Days of Summer (2009),*batteries not included (1987),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),13 Going on 30 (2004),1408 (2007),16 Blocks (2006),...,"Wizard of Oz, The (1939)","World Is Not Enough, The (1999)",Wreck-It Ralph (2012),"X-Files: Fight the Future, The (1998)",X-Men (2000),X-Men: Days of Future Past (2014),X-Men: First Class (2011),Zodiac (2007),Zombieland (2009),Zoolander (2001)
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
6,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [13]:
frequent_itemsets = apriori(dataset, min_support=0.05, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

MemoryError: Unable to allocate 18.8 GiB for an array with shape (54747476, 8, 46) and data type bool

In [ ]:
frequent_itemsets[ (frequent_itemsets['length'] == 2) &
                   (frequent_itemsets['support'] >= 0.05) ]

,support,itemsets,length
166,0.058824,"(Braveheart (1995), 10,000 BC (2008))",2
167,0.058824,"(Dead Poets Society (1989), 10,000 BC (2008))",2
168,0.058824,"(10,000 BC (2008), Gods Must Be Crazy II, The ...",2
169,0.058824,"(Hero (1992), 10,000 BC (2008))",2
170,0.058824,"(Killer, The (Die xue shuang xiong) (1989), 10...",2
...,...,...,...
1143,0.058824,"(Who Framed Roger Rabbit? (1988), There's Some...",2
1144,0.058824,"(Tombstone (1993), Top Gun (1986))",2
1145,0.058824,"(Truth About Cats & Dogs, The (1996), Tombston...",2
1146,0.058824,"(Truth About Cats & Dogs, The (1996), Top Gun ...",2


In [ ]:
frequent_itemsets[ (frequent_itemsets['length'] == 3) ].head()

,support,itemsets,length
1148,0.058824,"(Braveheart (1995), 10,000 BC (2008), Dead Poe...",3
1149,0.058824,"(Braveheart (1995), 10,000 BC (2008), Gods Mus...",3
1150,0.058824,"(Braveheart (1995), Hero (1992), 10,000 BC (20...",3
1151,0.058824,"(Braveheart (1995), Killer, The (Die xue shuan...",3
1152,0.058824,"(Braveheart (1995), Lion King, The (1994), 10,...",3


In [ ]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.2)
rules["antecedents_length"] = rules["antecedents"].apply(lambda x: len(x))
rules["consequents_length"] = rules["consequents"].apply(lambda x: len(x))
rules.sort_values("lift",ascending=False)

**Damped Mean**

In [ ]:
#Movive mean rating
ratings2 = pd.read_csv('ratings.csv')
mean_rating = ratings2.groupby('movieId')['rating'].mean().sort_values(ascending=False)
mean_rating

movieId
318     4.364362
858     4.315848
1248    4.259259
2959    4.258503
7502    4.247423
          ...   
1453    1.642857
1707    1.596491
2383    1.573171
3593    1.456140
66      1.153846
Name: rating, Length: 2500, dtype: float64

In [ ]:
#calculate Dampet Mean Rating
C = mean_rating.mean()  #mean of all mean ratings   
m = mean_rating.quantile(0.9) #90th percentile of mean ratings  
def weighted_rating(x, m=m, C=C):
    v = x['count']
    R = x['mean']
   
    return ((R*v + C*m) / (v+m)) #(v/(v+m) * R) + (m/(m+v) * C) #چرا اینا با هم برابرند؟

       

mean_rating = mean_rating.reset_index()
mean_rating['count'] = ratings2.groupby('movieId')['rating'].count()
mean_rating['mean'] = ratings2.groupby('movieId')['rating'].mean()
mean_rating['weighted_rating'] = mean_rating.apply(weighted_rating, axis=1)
mean_rating = mean_rating.sort_values('weighted_rating', ascending=False)
mean_rating.head()


,movieId,rating,count,mean,weighted_rating
318,2918,3.878613,564.0,4.364362,4.357477
858,46723,3.634375,448.0,4.315848,4.307616
1203,2355,3.480144,189.0,4.246032,4.228165
296,1288,3.891705,613.0,4.217781,4.212374
1221,85,3.472222,325.0,4.218462,4.208311
